In [ ]:
# データの永続化
from google.colab import drive 
drive.mount('/content/drive')
!mkdir -p '/content/drive/My Drive/work/'
%cd '/content/drive/My Drive/work/'

In [ ]:
# ソースからのHuggingface Transformersのインストール
!git clone https://github.com/huggingface/transformers -b v4.4.2
!pip install -e transformers
!pip install fugashi[unidic-lite]
!pip install ipadic

In [ ]:
# メニュー「ランタイム → ランタイムを再起動」で「Google Colab」を再起動

# 作業フォルダに移動
%cd '/content/drive/My Drive/work/'

In [ ]:
# Huggingface Datasetsのインストール
!pip install datasets==1.2.1

In [ ]:
import torch
from transformers import BertJapaneseTokenizer, AutoModelForMaskedLM

# トークナイザーとモデルの準備
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
model = AutoModelForMaskedLM.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

# マスク付きテキストの準備
text = f'「{tokenizer.mask_token}」っていう名前の妹が噂になってる'

# テキストをテンソルに変換
input_ids = tokenizer.encode(text, return_tensors='pt')

# マスクのインデックスの取得
masked_index = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]

# 推論
result = model(input_ids)
pred_ids = result[0][:, masked_index].topk(5).indices.tolist()[0]
for pred_id in pred_ids:
    output_ids = input_ids.tolist()[0]
    output_ids[masked_index] = pred_id
    print(tokenizer.decode(output_ids))

In [ ]:
%%time

# ファインチューニングの実行
!python ./transformers/examples/language-modeling/run_mlm.py \
    --model_type=bert \
    --model_name_or_path=cl-tohoku/bert-base-japanese-whole-word-masking \
    --train_file=train.txt \
    --validation_file=dev.txt \
    --do_train \
    --do_eval \
    --num_train_epochs=500 \
    --save_steps=5000 \
    --save_total_limit=3 \
    --output_dir=output/ \
    --use_fast_tokenizer=False \
    --logging_steps=100

In [ ]:
# 学習状況の確認
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
import torch
from transformers import BertJapaneseTokenizer, AutoModelForMaskedLM

# トークナイザーとモデルの準備
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
model = AutoModelForMaskedLM.from_pretrained('output/')

# マスク付きテキストの準備
text = f'「{tokenizer.mask_token}」っていう名前の妹が噂になってる'

# テキストをテンソルに変換
input_ids = tokenizer.encode(text, return_tensors='pt')

# マスクのインデックスの取得
masked_index = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]

# 推論
result = model(input_ids)
pred_ids = result[0][:, masked_index].topk(5).indices.tolist()[0]
for pred_id in pred_ids:
    output_ids = input_ids.tolist()[0]
    output_ids[masked_index] = pred_id
    print(tokenizer.decode(output_ids))